In [18]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv

# CSVファイルの設定
csv_file = 'hotels.csv'
csv_columns = ["ホテル名", "詳細ページURL", "アクセス情報", "価格情報", "一人あたり価格", "ホテルの評価", "レビュー数"]

# CSVファイルにヘッダーを書き込む
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=csv_columns)
    writer.writeheader()

base_url = "https://www.jalan.net/ikisaki/map/tokyo/"
response = requests.get(base_url)

soup = BeautifulSoup(response.content, "html.parser")
meta_tag = soup.find("meta", {"charset": True})

if meta_tag and meta_tag.get("charset"):
    response.encoding = meta_tag.get("charset")
else:
    response.encoding = response.apparent_encoding

soup = BeautifulSoup(response.text, "html.parser")

map_right_inner_div = soup.find("div", class_="map-right-inner")
if map_right_inner_div:
    a_tags = map_right_inner_div.find_all("a", class_="sub")
    print(f"発見した a タグの数: {len(a_tags)}")
else:
    print("対象の div が見つかりませんでした。")
    a_tags = []

urls = [urljoin(base_url, a['href']) for a in a_tags if 'href' in a.attrs]

def fix_encoding(text):
    try:
        return text.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        return text  # 修正できない場合そのまま返す

def get_hotel_data_from_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    meta_tag = soup.find("meta", {"charset": True})
    if meta_tag and meta_tag.get("charset"):
        response.encoding = meta_tag.get("charset")
    else:
        response.encoding = response.apparent_encoding
    soup = BeautifulSoup(response.text, "html.parser")

    hotel_divs = soup.find_all("div", class_="p-yadoCassette__body p-searchResultItem__body")
    hotels = []

    for hotel_div in hotel_divs:
        # ホテル名
        h2_tag = hotel_div.find("h2", class_="p-searchResultItem__facilityName")
        raw_hotel_name = h2_tag.get_text(strip=True) if h2_tag else "ホテル名なし"
        hotel_name = fix_encoding(raw_hotel_name)

        # 詳細ページURL
        a_tag = hotel_div.find("a", class_="jlnpc-yadoCassette__link s16_00 fb")
        detail_url = a_tag['data-href'] if a_tag and 'data-href' in a_tag.attrs else "URLなし"

        # アクセス情報
        access_tag = hotel_div.find("dd", class_="p-searchResultItem__accessValue")
        access_info = access_tag.get_text(strip=True) if access_tag else "アクセス情報なし"

        # 価格情報
        price_tag = hotel_div.find("span", class_="p-searchResultItem__lowestPriceValue")
        price_info = price_tag.get_text(strip=True) if price_tag else "価格情報なし"

        # 一人あたり価格
        unit_price_tag = hotel_div.find("span", class_="p-searchResultItem__lowestUnitPrice")
        unit_price_info = unit_price_tag.get_text(strip=True) if unit_price_tag else "一人あたり価格情報なし"

        # ホテルの評価
        rating_tag = hotel_div.find("span", class_="p-searchResultItem__summaryaverage-num")
        rating_info = rating_tag.get_text(strip=True) if rating_tag else "評価なし"

        # レビュー数
        review_count_tag = hotel_div.find("span", class_="ji ji-bubble-outline")
        review_count_info = review_count_tag.get_text(strip=True) if review_count_tag else "レビュー数なし"

        hotels.append({
            "ホテル名": hotel_name,
            "詳細ページURL": detail_url,
            "アクセス情報": access_info,
            "価格情報": price_info,
            "一人あたり価格": unit_price_info,
            "ホテルの評価": rating_info,
            "レビュー数": review_count_info
        })
    
    return hotels, soup

def get_next_page_url(current_url, page_number):
    next_url = f"{current_url}page{page_number}.html"
    return next_url

total_hotels_count = 0
for i, url in enumerate(urls, 1):
    print(f"URL {i}: {url}")
    current_url = url
    page_number = 2
    previous_page_title = None

    while current_url:
        print(f"現在のページ: {current_url}")
        hotels, soup = get_hotel_data_from_page(current_url)
        
        # ページタイトルが前のページと同じかどうかを確認
        current_page_title = soup.title.string if soup.title else None
        if current_page_title == previous_page_title:
            print("同じタイトルのページが検出されました。停止します。")
            break

        # 最後のページ以外のホテル情報を抽出
        if previous_page_title is not None:
            # CSVファイルに保存
            with open(csv_file, 'a', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=csv_columns)
                for hotel in hotels:
                    writer.writerow(hotel)

            total_hotels_count += len(hotels)

        previous_page_title = current_page_title
        current_url = get_next_page_url(url, page_number)
        page_number += 1

    print(f"URL {i} で取得したホテルの数: {total_hotels_count}")

print(f"総取得ホテル数: {total_hotels_count}")

発見した a タグの数: 13
URL 1: https://www.jalan.net/130000/LRG_137100/
現在のページ: https://www.jalan.net/130000/LRG_137100/
現在のページ: https://www.jalan.net/130000/LRG_137100/page2.html
現在のページ: https://www.jalan.net/130000/LRG_137100/page3.html
現在のページ: https://www.jalan.net/130000/LRG_137100/page4.html
現在のページ: https://www.jalan.net/130000/LRG_137100/page5.html
同じタイトルのページが検出されました。停止します。
URL 1 で取得したホテルの数: 91
URL 2: https://www.jalan.net/130000/LRG_138000/
現在のページ: https://www.jalan.net/130000/LRG_138000/
現在のページ: https://www.jalan.net/130000/LRG_138000/page2.html
現在のページ: https://www.jalan.net/130000/LRG_138000/page3.html
現在のページ: https://www.jalan.net/130000/LRG_138000/page4.html
現在のページ: https://www.jalan.net/130000/LRG_138000/page5.html
現在のページ: https://www.jalan.net/130000/LRG_138000/page6.html
同じタイトルのページが検出されました。停止します。
URL 2 で取得したホテルの数: 206
URL 3: https://www.jalan.net/130000/LRG_136200/
現在のページ: https://www.jalan.net/130000/LRG_136200/
現在のページ: https://www.jalan.net/130000/LRG_136200/page2.html
現在のページ: 